In [4]:
# Libraries
import numpy as np
import pandas as pd
import os

In [21]:
# Load Data set and check Shape
data = ("../data/raw/e_commerce_shopper_behaviour_and_lifestyle.csv")
ecom_df = pd.read_csv(data)
ecom_df.shape

(1000000, 60)

We can see that there are 60 colomuns and 1M data points under each row.

In [30]:
# Show all column names
print(ecom_df.columns.tolist())

# Column names with data types
print(ecom_df.dtypes)

['user_id', 'age', 'gender', 'country', 'urban_rural', 'income_level', 'employment_status', 'education_level', 'relationship_status', 'has_children', 'household_size', 'occupation', 'ethnicity', 'language_preference', 'device_type', 'weekly_purchases', 'monthly_spend', 'cart_abandonment_rate', 'review_writing_frequency', 'average_order_value', 'preferred_payment_method', 'coupon_usage_frequency', 'loyalty_program_member', 'referral_count', 'product_category_preference', 'shopping_time_of_day', 'weekend_shopper', 'impulse_purchases_per_month', 'browse_to_buy_ratio', 'return_frequency', 'budgeting_style', 'brand_loyalty_score', 'impulse_buying_score', 'environmental_consciousness', 'health_conscious_shopping', 'travel_frequency', 'hobby_count', 'social_media_influence_score', 'reading_habits', 'exercise_frequency', 'stress_from_financial_decisions', 'overall_stress_level', 'sleep_quality', 'physical_activity_level', 'mental_health_score', 'daily_session_time_minutes', 'product_views_per_

In [29]:
for col in ecom_df.select_dtypes(include='object').columns:
    print(f"\n{col}:")
    print(ecom_df[col].unique())
    print(f"Count: {ecom_df[col].nunique()}")


gender:
['Female' 'Male' 'Non-binary' 'Other']
Count: 4

country:
['Germany' 'Japan' 'India' 'Canada' 'USA' 'France' 'Australia' 'UK'
 'Brazil' 'China']
Count: 10

urban_rural:
['Suburban' 'Urban' 'Rural']
Count: 3

employment_status:
['Self-employed' 'Unemployed' 'Employed' 'Student' 'Retired']
Count: 5

education_level:
['Associate Degree' 'Bachelor' 'High School' 'Master' 'PhD']
Count: 5

relationship_status:
['Single' 'Married' 'Widowed' 'In a relationship' 'Divorced']
Count: 5

occupation:
['Healthcare' 'Finance' 'Engineering' 'Other' 'Marketing' 'Retail' 'IT'
 'Education']
Count: 8

ethnicity:
['Other' 'Hispanic' 'Asian' 'Caucasian' 'African American']
Count: 5

language_preference:
['English' 'Mandarin' 'Hindi' 'German' 'French' 'Spanish']
Count: 6

device_type:
['Mobile' 'Desktop' 'Tablet']
Count: 3

preferred_payment_method:
['PayPal' 'Google Pay' 'Credit Card' 'Apple Pay' 'Debit Card'
 'Bank Transfer']
Count: 6

product_category_preference:
['Groceries' 'Beauty' 'Books' 'Toy

Let us restrict country to USA only as it helps us focus easiloy and lets the model not over fit with data.

In [33]:
# Filter for USA only and drop country column
usa_df = ecom_df[ecom_df['country'] == 'USA'].drop('country', axis=1)

# Verify
print(f"Original shape: {ecom_df.shape}")
print(f"USA only shape: {usa_df.shape}")


Original shape: (1000000, 60)
USA only shape: (99996, 59)


As a result 900,004 values have been eliminated as our Focus group is USA only which helps in focusing and creating a better model